In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.insert(0,'../../modules')

In [2]:
import numpy as np
from maze_problem import Maze

# Markov Decision Processes
Many problems require making a set of sequential decisions (e.g chess, go, card games). A sensible way to frame these problems in in terms of actions ($A$), states ($S$), and rewards ($R$). At each timestep an agent takes an action that transitions the current state into a new state with a given probability. When taking an action in a state the agent also recieves a reward for doing so (e.g jumping at prey). We refer to the policy an agent takes at each step (which action to choose) as $\pi$. <br>
**Example**: Solving a maze <br> 
The states could be represented as positions the player can be in. Actions are which direction to move in and the reward is high for being in the finishing state and -1 elsewhere (There is a cost to taking a long time to solve a maze). <br>

In general it is computationally intractable to solve these kinds of problems (find the best actions to take in each state). This is because the probability of being in a new state depends on the entire history of states. This means making a decision requires searching all possible trajectories. <br>
In a Markov Decision Process (MDP) we make things tractable by assuming that the probability of the next state depends only on the current state and action. We also say there is a finite set of states and actions to choose from. The the probability of the next state is given by the function $T(s'|s,a)$

For a MDP problem with a finite number of timesteps $n$ the total utility is given as the sum of rewards at all timesteps:
$$U^\pi = \sum_{i=1}^n r_i$$
Here $r_i$ refers to the particular reward given but in general rewards have a probability so we use $R(s,a)$ to refer to the reward for being in state $s$ and executing action $a$. For an infinite horizon problem the above becomes infinite so we use a lambda discount term:
$$U^\pi = \sum_{i=1}^\infty \lambda^{i-1} R(s_i,a_i)$$
An alternative is to use a mean of rewards but that can be computationally difficult. As we start in a particular state we say $U^\pi(s)$ is the utility we expect given starting in state $s$. $s_i$ refers to the $i'th$ relative to $s$ (so, $s_1$ is the same as $s$ and $s_2$ is the next step otherwise called $s'$). The same notation is used for actions.

If we write the function $$ U_m^\pi(s) = \sum_{i=1}^m \lambda^{i-1} R(s_i,a_i)$$
(So, the evaluation of the utility up to a certain point) then we can see $U_\infty^\pi$ is the total utility for starting in state $s$.

We can also then write
$$
\begin{aligned}
    U_{m+1}^\pi(s) &= \sum_{i=1}^{m+1} \lambda^{i-1} R(s_i,a_i) \\
                &= R(s,a) + \lambda \sum_{i=2}^{m+1} \lambda^{i-1} R(s_i,a_i) \\
\end{aligned}
$$
By pulling the first reward out. The second part of the equation is now just the utility up to $m$ from starting in the next state: <br> <br>
$$U_{m+1}^\pi(s) = R(s,a) + \lambda U_{m}^\pi(s')$$ <br>
As we don't actually know what the next state will be we have to take the expectation of $U_m^\pi$: <br> <br>
$$U_{m+1}^\pi(s) = R(s,a) + \lambda \sum_{s'} T(s'|s,a) U_{m}^\pi(s')$$ <br>
This is the lookahead equation for a markov decision process. 

With an infinite time series the best decision to make becomes solely dependent on the current state. This is not the case for a finite time series where the actions later in a game will look for shorter term gains. Thus $T$ becomes a matrix $T(s'|s)$ as the actions are deterministic. 

Then the values for each state (called the value function) can be written down in vector form: <br>
$$U = R +\lambda T \cdot U$$
Which can be solved:
$$
\begin{aligned}
    U &= R +\lambda T \cdot U \\
    U - \lambda T \cdot U &= R \\
    (I - \lambda T) \cdot U &= R \\
    U &= (I - \lambda T)^{-1} R \\
\end{aligned}
$$

### Evaluating a simple (actually became very complicated) decision:
**W** is a wall <br>
**F** is a fire <br>
**G** is gold <br>
**B** is empty <br>
**S** is start. <br>
Actions are moving in a direction. Moving into a wall leaves you in the same place. <br>
You have a $70\%$ chance of going the way you choose to move, $15\%$ chance of slipping and falling to the left and also $15\%$ chance of falling to the right. <br>
The possible states are just where you are.

In [3]:
world = np.array([['W','W','W','W','W'],
                  ['W','B','B','G','W'],
                  ['W','B','F','B','W'],
                  ['W','S','F','B','W'],
                  ['W','B','B','B','W'],
                  ['W','W','W','W','W']])

maze = Maze(world)
print(maze)

[[W   W   W   W   W ] 
 [W   B   B   G   W ] 
 [W   B   F   B   W ] 
 [W   S   F   B   W ] 
 [W   B   B   B   W ] 
 [W   W   W   W   W ]] world map
[[                  ] 
 [    0   1   2     ] 
 [    3   4   5     ] 
 [    6   7   8     ] 
 [    9   10  11    ] 
 [                  ]] state map


**A good policy:** avoid fire, go to gold

In [4]:
good_policy = ['R','R','R','U','U','U','U','R','U','R','R','U']
good_policy_transition_matrix = maze.get_policy_matrix(good_policy)
maze.make_animation(good_policy_transition_matrix,20)

**A bad policy:** go to fire.

In [5]:
bad_policy = ['R','D','L','R','D','L','R','U','L','U','U','L']
bad_policy_transition_matrix = maze.get_policy_matrix(bad_policy)
maze.make_animation(bad_policy_transition_matrix,20)

### Infinite Case

In [6]:
def calculate_value_function(reward,lambda_discount,transition_matrix):
    # transpose included here because I index current state in columns, next state in rows.
    return np.linalg.inv(np.eye(len(reward))-lambda_discount*transition_matrix.T).dot(reward) 

In [7]:
reward = np.array([-1,-1,99,-1,-30,-1,-1,-30,-1,-1,-1,-1])
discount = 0.95
maze.show_on_map(reward)

[[                  ] 
 [    -1  -1  99    ] 
 [    -1  -30 -1    ] 
 [    -1  -30 -1    ] 
 [    -1  -1  -1    ] 
 [                  ]]


In [8]:
good_values = calculate_value_function(reward,discount,good_policy_transition_matrix)
maze.show_on_map(good_values.round(1))

[[                          ] 
 [      221.9 244.0 274.2   ] 
 [      203.5 196.1 244.0   ] 
 [      184.4 166.9 215.8   ] 
 [      167.9 178.5 195.9   ] 
 [                          ]]


In [9]:
bad_values = calculate_value_function(reward,discount,bad_policy_transition_matrix)
maze.show_on_map(bad_values.round(1))

[[                              ] 
 [       -332.4 -348.7 -250.7   ] 
 [       -365.5 -398.0 -353.5   ] 
 [       -368.1 -399.3 -366.1   ] 
 [       -347.3 -365.2 -345.2   ] 
 [                              ]]


So, clearly the first policy is better (for the given reward function)

### Finite Case
We also always have the option of evaluating the utilities of a given policy one by one when the number of steps if finite. However this requires checking every possible trajectory. The best option for the first action requires calculating the value of all future actions and states. <br>
Say in the above problem you can make 4 steps.
In the general case where the action can depend on the current state *and* time:

In [10]:
#Say our policy is:
timestep1_policy = ["na","na","na","na","na","na","U","na","na","na","na","na"]
timestep2_policy = ["na","na","na","U","na","na","L","R","na","na","na","na"]
timestep3_policy = ["R","na","na","L","U","na","L","D","U","L","D","na"]
timestep4_policy = ["U","R","na","L","U","U","L","D","R","L","D","R"]

action_to_matrix = {'L':maze.left_transition_matrix,
                    'R':maze.right_transition_matrix,
                    'U':maze.up_transition_matrix,
                    'D':maze.down_transition_matrix}

print("policy step 1:")
maze.show_on_map(timestep1_policy)
print("")
print("policy step 2:")
maze.show_on_map(timestep2_policy)
print("")
print("policy step 3:")
maze.show_on_map(timestep3_policy)
print("")
print("policy step 4:")
maze.show_on_map(timestep4_policy)

policy step 1:
[[                  ] 
 [    na  na  na    ] 
 [    na  na  na    ] 
 [    U   na  na    ] 
 [    na  na  na    ] 
 [                  ]]

policy step 2:
[[                  ] 
 [    na  na  na    ] 
 [    U   na  na    ] 
 [    L   R   na    ] 
 [    na  na  na    ] 
 [                  ]]

policy step 3:
[[                  ] 
 [    R   na  na    ] 
 [    L   U   na    ] 
 [    L   D   U     ] 
 [    L   D   na    ] 
 [                  ]]

policy step 4:
[[                  ] 
 [    U   R   na    ] 
 [    L   U   U     ] 
 [    L   D   R     ] 
 [    L   D   R     ] 
 [                  ]]


In [11]:
rewards = []
probs = []

state1 = 6
action1 = timestep1_policy[state1]
transition1_prob = action_to_matrix[action1][:,state1]
for state2 in np.arange(12)[transition1_prob>0]:
    action2 = timestep2_policy[state2]
    transition2_prob = action_to_matrix[action2][:,state2]
    for state3 in np.arange(12)[transition2_prob>0]:
        action3 = timestep3_policy[state3]
        transition3_prob = action_to_matrix[action3][:,state3]
        for state4 in np.arange(12)[transition3_prob>0]:
            action4 = timestep4_policy[state4]
            transition4_prob = action_to_matrix[action4][:,state4]
            for state5 in np.arange(12)[transition4_prob>0]:
                p1 = transition1_prob[state2]
                p2 = transition2_prob[state3]
                p3 = transition3_prob[state4]
                p4 = transition4_prob[state5]
                r1 = reward[state1]
                r2 = reward[state2]
                r3 = reward[state3]
                r4 = reward[state4]
                r5 = reward[state5]
                rewards.append(r1+r2+r3+r4+r5)
                probs.append(p1*p2*p3*p4)

assert np.isclose(np.sum(probs),1)
expected_utility = np.sum(np.array(probs)*np.array(rewards))
print("expected utility",expected_utility.round(3))

expected utility 20.954


However, there is also a dynamic programming approach which is far quicker: <br>
define:
$$U_m^\pi(s)=\sum_{i=1}^{m} R(s_i,a_i)$$
similarly to before $U_n^\pi$ is the final utility for making n steps. <br>
We can say:
$$
\begin{aligned}
    U_{m+1}^\pi(s)&=\sum_{i=1}^{m+1} R(s_i,a_i) \\
    U_{m+1}^\pi(s)&= R(s,a) + \sum_{s'} T(s'|s,a) U_m^\pi(s')
\end{aligned}
$$
If you think just in terms of matricies for 3 steps this is:
$$R_1 + T_1 \cdot (R_2 + T_2 \cdot (R_3))$$
which can be represented on a graph easily. E.g:

<img src="Images/graph1.jpg" width="500" align="left">

This problem can be solved with dynamic programming:

<img src="Images/graph2.jpg" width="300" align="left">

<img src="Images/graph3.jpg" width="150" align="left">

Getting $T_i$

In [12]:
T1 = maze.get_policy_matrix(timestep1_policy)
T2 = maze.get_policy_matrix(timestep2_policy)
T3 = maze.get_policy_matrix(timestep3_policy)
T4 = maze.get_policy_matrix(timestep4_policy)

rewards are the same at each step, so just run multiplication:

In [13]:
utilities = reward + T1.T.dot(reward+T2.T.dot(reward+T3.T.dot(reward+T4.T.dot(reward))))
print("expected utility via dynamic approach",utilities[6].round(3))

expected utility via dynamic approach 20.954
